# Quantum Machine Learning. Regressor

Quantum Machine Learning is one the the possible applications of Quantum Computing. This exercise is based on the paper [Mitarai K, Negoro M, Kitagawa M, Fujii K. Quantum circuit learning. Phys Rev A. 2018;98(3):032309.](https://arxiv.org/abs/1803.00745)

The example is inspired on [QCL page](http://dkopczyk.quantee.co.uk/qcl/), where a good description of the method is provided. However, to speedup the execution, instead the usual gradient descend algorithm for Machine Learning, a faster optimisation algorithm is used [Conjugate Gradient](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin_cg.html)

The steps of the algorithm are:

1. Starting from state $|0\rangle$ and an case with input data ${x_i}$ of the dataset ($X$,$Y$), generate an initial Quantun State $|\Psi({x_i})>$ on the desired number of qubits (which can be larger than the number of input data for a single case). The amplitudes of this Quantum State are the input quantum features of the Machine Learning Model.
2. Apply $n$ times a combination of a random Time Evolution of one operator plus a set of parameterised rotations on each qubits.
3. Measure the expectation value of one operator of subset of Pauli operators $\{B_j\} \subset {\{I,X,Y,Z\}}^{\otimes N}$. This measurement will be the prediction $y_i$ for this input data ${x_i}$.
4. Minimize the cost function which compares the labels $Y$ with the predictions $y$.


Import the needed operations from ProjectQ

In [ ]:
from projectq.cengines import MainEngine
from projectq.backends import Simulator
from projectq.ops import Z,X,H,Rx,Ry,Rz,All,Measure

Generate a dataset which follows the funcion $f(x)=x^2$ on the interval $[-1,1]$. This restriction on the input values does not limit the model, because in Classical Machine Learning, usually the input and output data are normalised as a first step.
Select the number of cases of the dataset assigning a value to P (take into account that a large number means long simulations)

In [ ]:
import numpy as np
P=20
data_in=np.linspace(-1,1,P)
data_out=data_in*data_in

To inicialise the state (step 1), apply $R_z(cos^{-1}(x^2))R_y(sin^{-1}(x))$ to all qubits of the Quantum Register. Any other inicialisation is possible. If N i sthe number of qubits, the new state is:

$$|\Psi(x)>= (R_z(cos^{-1}(x^2))R_y(sin^{-1}(x))|0>)^{\otimes N}$$

In [ ]:
def Uinit(qreg,x):
    import math
    theta_1=math.acos(x*x)
    theta_2=math.asin(x)
    All(Ry(theta_2)) | qreg
    All(Rz(theta_1)) | qreg


Create now the Unitary operation **U** to train. It receives as input the qubits in a Quantum Register (*qreg*) and an array with the parameters of the rotations which define this unitary operation.

For this case, to **each** qubit apply three rotations

$$R_x(\theta_{1j})R_z(\theta_{2j})R_x(\theta_{3j})$$ 

where $j$ is the index of the qubit. So, $\Theta =\{\theta_{ij}\}$ is an array of paramenters to train of dimensions $[3,N]$, where N is the number of qubits

In [ ]:
def U(qreg,theta,d):
    for index,q in enumerate(qreg):
            Rx(theta[0,index,d]) | q
            Rz(theta[1,index,d]) | q
            Rx(theta[2,index,d]) | q

Before this Unitary operation, the state will evolve during a selected time **T** using a Hamiltonian which follows a fully connected transverse Isin model:

$$H = \sum_{j=1}^{N}\alpha_j X_j + \sum_{j=1}^{N}\sum_{k=1}^{j-1}J_{jk}Z_jZ_k$$

where $N$ is the number of qubits. 

In [ ]:
from projectq.ops import QubitOperator

In [ ]:
def Circuit_Hamiltonian(N,alpha,J):
    Hamiltonian=None
    k=0
    for j in range(N):
        if (Hamiltonian is None):
            Hamiltonian=alpha[j]*QubitOperator("X%d"%j)
        else:
            Hamiltonian=Hamiltonian+alpha[j]*QubitOperator("X%d"%j)

        for i in range(j):
            Hamiltonian=Hamiltonian+J[k]*QubitOperator("Z%d Z%d"%(j,i))
            k=k+1
    return Hamiltonian


In [ ]:
from projectq.ops import TimeEvolution

Now it is time to compose the final circuit. The graphical representation of the circuit is:

<img src="Images/QCL.jpg" width="60%"/>

The function receives:
1. **x** that is the input data for this case
2. **Theta** a vector the dimension 3\*N\*D+1. Last element multiplies the expectation value before exit the function.
3. **N**, the number of qubits 
4. **D**, the number of times to repeat the evolution
5. **T**, the time to evolve th Hamiltonian
6. **H**, the Hamiltonian

The output must be the expectation value of $\sigma_z$ for qubit **0** multiplied by the parameter Theta\[-1\]

Hint: Use the ProjectQ methods:
1. [QubitOperator](https://projectq.readthedocs.io/en/latest/projectq.ops.html#projectq.ops.QubitOperator)
2. [TimeEvolution](https://projectq.readthedocs.io/en/latest/projectq.ops.html#projectq.ops.TimeEvolution)
3. [get_expectation_value](https://projectq.readthedocs.io/en/latest/projectq.backends.html#projectq.backends.Simulator)

In [ ]:
def qfun(x,Theta,N,D,T,H):
    theta1=Theta[0:3*N*D].reshape(3,N,D)

    eng=MainEngine(backend=Simulator(gate_fusion=True, rnd_seed=1000))
    qreg=eng.allocate_qureg(N)  
    
    Uinit(qreg,x)
    
    for i in range(D):
        TimeEvolution(time=T,hamiltonian=H) | qreg
        U(qreg,theta1,i)
    
    eng.flush()
    
    energy=eng.backend.get_expectation_value(QubitOperator("Z0"),[qreg[0]])
    
    All(Measure) | qreg
    eng.flush()
    del eng
    
    return Theta[-1]*energy
    

OK. Define the loss for the training:

$$loss = \frac{1}{m}\sum_{i=1}^m{(qfun(x_i)-y_i)}^2$$

where $m$ is the number of cases and $y_i$ is the label for case $i$

In [ ]:
def loss(Theta,data_in,data_out,N,D,T,H):
    fun_out=0
    for i in range(len(data_in)):
        x=data_in[i]
        y=data_out[i]
        y_pred=qfun(x,Theta,N,D,T,H)
        fun_out+=(y_pred-y)**2
        
    return fun_out/len(data_in)

In [ ]:
def jacobian(Theta,data_in,data_out,N,D,T,H):
    import random
    import math
    fun_out=np.zeros(len(data_in))
    for i in range(len(data_in)):
        x=data_in[i]
        y=data_out[i]
        y_pred=qfun(x,Theta,N,D,T,H)
        fun_out[i]=(y_pred-y)
    grad=np.zeros(len(Theta))
    for j in range(len(Theta)-1):
        grad[j]=0.
        for i in range(len(data_in)):
            x=data_in[i]
            Theta_in=Theta.copy()
            Theta_in[j]=Theta[j]+(math.pi*0.5)
            bplus=qfun(x,Theta_in,N,D,T,H)
            Theta_in[j]=Theta[j]-(math.pi*0.5)
            bminus=qfun(x,Theta_in,N,D,T,H)
            grad[j]+=(bplus-bminus)*fun_out[i]
    """
    The last weight is special, because it is not a rotation.
    """
    grad[-1]=0.
    for i in range(len(data_in)):
        x=data_in[i]
        grad[-1]+=2.*fun_out[i]*qfun(x,Theta,N,D,T,H)
    grad[-1]=grad[-1]/Theta[-1]
    grad=grad/len(data_in)
    return grad


Initialise the training parameters. The angles of the rotations must be selected randomly in the interval $[0,2\pi]$, meanwhile the factor of the expectation value is initialised to 1.0

Select also the number of repetitions (D) and the time for the Hamiltonian evolution (T)

**Hint**: if you want to speedup the convergence, use these pre-trained values for N=3, D=2 and T=10.

alpha= np.array([-0.09437294,  0.90506988, -0.67730307 ])

J= np.array([-0.62281193, -0.59477404, -0.24760364])

Theta= np.array([ 1.67414239, 5.81058199, 5.48748225,  4.76538875,  0.34889076,  5.14003758,
  6.46545405,  4.59147751,  4.80588215,  4.51748371,  3.73661614,  5.55503066,
  2.32459032,  1.04557867,  4.48241849,  5.51983066, -0.33860855,  5.99787863,
  2.00313909])

 and select the number of iterations to 10


In [ ]:
N=3
D=2
T=10
alpha=(np.random.rand(N)-0.5)*2.0
J=(np.random.rand(N*(N-1)//2)-0.5)*2.0
Theta=np.random.rand(3*N*D+1)
Theta[0:3*N*D]=np.pi*2.*Theta[0:3*N*D]
Theta[-1]=1.0
"""
UNCOMMENT TO SPEEP-UP

alpha= np.array([ -0.09437294,  0.90506988, -0.67730307 ])
J= np.array([-0.62281193,-0.59477404, -0.24760364])
Theta= np.array([ 1.67414239, 5.81058199, 5.48748225,  4.76538875,  0.34889076,  5.14003758,
  6.46545405,  4.59147751,  4.80588215,  4.51748371,  3.73661614,  5.55503066,
  2.32459032,  1.04557867,  4.48241849,  5.51983066, -0.33860855,  5.99787863,
  2.00313909])
"""
Hamiltonian=Circuit_Hamiltonian(N,alpha,J)
print("The value of the initial loss is =",loss(Theta,data_in,data_out,N,D,T,Hamiltonian))

It is possible to check that the Jacobian is correct, comparing the result against the numerical gradient for the first point

In [ ]:
grad=jacobian(Theta,data_in[0:1],data_out[0:1],N,D,T,Hamiltonian)
h=0.000001
for i in range(len(Theta)):
    Theta_in=Theta.copy()
    Theta_in[i]=Theta[i]+h
    h1=loss(Theta_in,data_in[0:1],data_out[0:1],N,D,T,Hamiltonian)
    Theta_in[i]=Theta[i]-h
    h2=loss(Theta_in,data_in[0:1],data_out[0:1],N,D,T,Hamiltonian)
    grad2=(h1-h2)/(2*h)
    print("Grad[%d]:%.5f - NumericalGrad[%d]:%.5f"%(i,grad[i],i,grad2))

Plot the initial form of the proposed function.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
out=[qfun(j,Theta,N,D,T,Hamiltonian) for j in data_in]
plt.plot(data_in,out,label="Init model")
plt.xlabel("X")
plt.plot(data_in,data_out,"*",label="Labels")
plt.legend()
plt.show()

This is a callback function to show how the training evolves. **Just execute the next cell without changes**

In [ ]:
def callback(xk):
    cost=loss(xk,data_in,data_out,N,D,T,Hamiltonian)
    out=[qfun(j,xk,N,D,T,Hamiltonian) for j in data_in]
    plt.plot(data_in,out,"b",label="Cost=%.4f"%cost)
    plt.plot(data_in,data_in*data_in,"*",label="Labels")
    out=[qfun(j,Theta,N,D,T,Hamiltonian) for j in data_in]
    plt.plot(data_in,out,"r",label="First")
    plt.xlabel("X")
    plt.legend()
    plt.show()
    print("Cost=",cost)
    print("Theta=",xk)
    return False

Ok. Now train the model. Select the number of iterations in maxiter

Use the minimize method with "Conjugate Gradient (CG)" as algorithm of minimisation. Do not forget to add the callback, so you can follow the training evolution.

**Hint**: [Minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)

In [ ]:
from scipy.optimize import minimize
import math
options={'maxiter': 40, 'disp': True}
args=(data_in,data_out,N,D,T,Hamiltonian)
Theta_out=minimize(loss,Theta,args=args, jac=jacobian,callback=callback,method='CG', options=options)

Show the final results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#qfun(j,Theta,N,D,T)
out=[qfun(j,Theta_out.x,N,D,T,Hamiltonian) for j in data_in]
plt.plot(data_in,out,"b",label="QML model")
plt.plot(data_in,data_in*data_in,"*", label="Labels")
out=[qfun(j,Theta,N,D,T,Hamiltonian) for j in data_in]
plt.plot(data_in,out,"r",label="Initial model")
plt.legend()
plt.xlabel("X")
plt.show()

Remember to store all the parameters that define this model.

In [ ]:
print("alpha=",alpha)

In [ ]:
print("J=",J)

In [ ]:
print("Theta=",Theta_out.x)